# Prepare the data

In [69]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [70]:
words = open('names.txt', 'r').read().splitlines()
len(words)

32033

In [71]:
chars = sorted(list(set("".join(words))))
vocab_size = len(chars)

In [72]:
stoi = {char:i for i, char in enumerate(chars)}
stoi['.'] = 0 
itos = {i: char for char, i  in stoi.items()}

In [73]:
block_size = 3 
X, Y = [], []

for w in words[:5]:
    context = [0]*block_size
    for ch in w+'.':
        ix = stoi[ch]
        Y.append(ix)
        X.append(context)
        context = context[1:] + [ix]
        
X = torch.tensor(X)
Y = torch.tensor(Y)
len(chars)

26

# MLP

In [74]:
# create an embeding matrix
# 26 x 20 
E = torch.randn((27, 4), requires_grad=True)
emb = E[X]
emb.shape[0]


32

In [75]:
W1 = torch.randn((12, 100))
b1 = torch.randn(100)

In [76]:
h = torch.tanh(emb.view(-1, 12) @ W1 + b1)
h.shape


torch.Size([32, 100])

In [77]:
E = torch.randn((27, 4), requires_grad=True)
W1 = torch.randn((12, 100))
b1 = torch.randn(100)
W2 = torch.randn((100, 27))
b2 = torch.randn(27)
parameters = [E, W1, W2, b1, b2]

In [80]:
params = sum(p.nelement() for p in parameters)
params

4135

In [81]:
emb = E[X]
h = torch.tanh(emb.view(-1, 12) @ W1 + b1)
logits = h @ W2 +b2
loss = F.cross_entropy(logits, Y)
loss

tensor(13.9957, grad_fn=<NllLossBackward0>)